In [ ]:
"""
Gapminder demo demonstrating how to combine to extend a HoloViews plot
with custom bokeh widgets to deploy an app.
"""

Might need to execute `bokeh.sampledata.download()` for next cell to work.

In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv

from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.layouts import layout
from bokeh.models import Slider, Button
from bokeh.sampledata import gapminder
from holoviews.plotting.bokeh import BokehRenderer

try:
    from jupyterlabutils import show_with_bokeh_server
except ModuleNotFoundError:
    from notebook_utils import show_with_bokeh_server

In [ ]:
output_notebook(INLINE, hide_banner=True)

In [ ]:
class gm_plot(object):
    def __init__(self, doc):
        # Declare dataset
        panel = pd.Panel({'Fertility': gapminder.fertility,
                          'Population': gapminder.population,
                          'Life expectancy': gapminder.life_expectancy})
        gapminder_df = panel.to_frame().reset_index().rename(columns={'minor': 'Year'})
        gapminder_df = gapminder_df.merge(gapminder.regions.reset_index(), on='Country')
        gapminder_df['Country'] = gapminder_df['Country'].astype('str')
        gapminder_df['Group'] = gapminder_df['Group'].astype('str')
        gapminder_df.Year = gapminder_df.Year.astype('f')
        ds = hv.Dataset(gapminder_df)

        # Apply dimension labels and ranges
        kdims = ['Fertility', 'Life expectancy']
        vdims = ['Country', 'Population', 'Group']
        dimensions = {
            'Fertility' : dict(label='Children per woman (total fertility)', range=(0, 10)),
            'Life expectancy': dict(label='Life expectancy at birth (years)', range=(15, 100)),
            'Population': ('population', 'Population')
        }

        # Create Points plotting fertility vs life expectancy indexed by Year
        gapminder_ds = ds.redim(**dimensions).to(hv.Points, kdims, vdims, 'Year')

        # Define annotations
        text = gapminder_ds.clone({yr: hv.Text(1.2, 25, str(int(yr)), fontsize=30)
                           for yr in gapminder_ds.keys()})

        # Define options
        opts = {'plot': dict(width=1000, height=600,tools=['hover'], size_index='Population',
                             color_index='Group', size_fn=np.sqrt, title_format="{label}"),
                'style': dict(cmap='Set1', size=0.3, line_color='black', alpha=0.6)}
        text_opts = {'style': dict(text_font_size='52pt', text_color='lightgray')}

        # Combine Points and Text
        hvgapminder = (gapminder_ds({'Points': opts}) * text({'Text': text_opts})).relabel('Gapminder Demo')
        
        self.doc = doc
        self.hvplot = BokehRenderer.get_plot(hvgapminder, self.doc)
        self.button = Button(label='► Play', width=60)
        self.button.on_click(self.animate)
        self.start, self.end = ds.range('Year')
        self.slider = Slider(start=self.start, end=self.end, value=self.start, step=1, title="Year")
        self.slider.on_change('value', self.slider_update)
        plot = layout([[self.hvplot.state], [self.slider, self.button]], sizing_mode='fixed')
        self.doc.add_root(plot)
        
    def slider_update(self, attrname, old, new):
        self.hvplot.update((new,))
        
    # Define custom widgets
    def animate_update(self):
        year = self.slider.value + 1
        if year > self.end:
            year = self.start
        self.slider.value = year

    def animate(self):
        if self.button.label == '► Play':
            self.button.label = '❚❚ Pause'
            self.doc.add_periodic_callback(self.animate_update, 200)
        else:
            self.button.label = '► Play'
            self.doc.remove_periodic_callback(self.animate_update)

show_with_bokeh_server(gm_plot)